In [1]:
import urllib.request
import urllib.parse
import urllib.error
import ssl
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import pandas as pd
import re

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [2]:
# extract information of houses on market 40 houses
market_houses = []
markethouse_url = 'https://www.zillow.com/newark-ca/houses/?searchQueryState={%22pagination%22:{},%22usersSearchTerm%22:%22Newark,%20CA%22,%22mapBounds%22:{%22west%22:-122.16536934228516,%22east%22:-121.89551765771485,%22south%22:37.436669756614464,%22north%22:37.60533202072703},%22regionSelection%22:[{%22regionId%22:49611,%22regionType%22:6}],%22mapZoom%22:12,%22filterState%22:{%22isCondo%22:{%22value%22:false},%22isMultiFamily%22:{%22value%22:false},%22isManufactured%22:{%22value%22:false},%22isLotLand%22:{%22value%22:false},%22isTownhouse%22:{%22value%22:false},%22isApartment%22:{%22value%22:false}},%22isListVisible%22:true,%22isMapVisible%22:true}'
market_req = Request(markethouse_url, headers = {'user-Agent':'Mozilla/5.0'})
market_text = urlopen(market_req).read().decode(encoding="utf-8", errors='ignore')
market_soup = BS(market_text, 'html.parser')
for link in market_soup.findAll('a',attrs = {'class':'list-card-link', 'tabindex':"0"}, href = True):
    market_houses.append(link['href'])


In [3]:
len(market_houses)

40

In [4]:
#create a empty dataframe
house_df = pd.DataFrame()

In [5]:
#informations on the first listing page of all 40 houses
bds = []
bath = []
areas = []
price_ls = []
adr_ls = []


req = Request(markethouse_url, headers = {'user-Agent':'Mozilla/5.0'})
text = urlopen(req).read().decode(encoding="utf8", errors='ignore')
soup = BS(text, 'html.parser')
    
house_prices = soup.findAll('div', attrs = {'class':'list-card-price'})
for price in house_prices:
    price_ls.append(price.get_text())


    
#extract room info
lists = soup.findAll('ul', attrs = {'class':'list-card-details'})
details = soup.findAll('ul', attrs = {'class':'list-card-details'})
#print(len(details))
for detail in details:
    #print(detail.li)
    bds.append(detail.li.get_text())
    bath.append(detail.li.next_sibling.get_text())
    areas.append(detail.li.next_sibling.next_sibling.get_text())
    
#extract address info
addresses = soup.findAll('address', attrs = {'class':'list-card-addr'})
for address in addresses:
    adr_ls.append(address.get_text())

In [6]:
len(bds)

40

In [7]:
#details after opening each url of a house

lotsize = []
year = []
ac = []
heat = []
park = []
price_sqft = []
hoa = []

elem = []
mid =[]
high=[]

elem_s=[]
mid_s=[]
high_s=[]

neighborhood = []

for url in market_houses:
    req = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text = urlopen(req).read().decode(encoding="utf-8", errors='ignore')
    soup = BS(text, 'html.parser')
    
    #iterate all house url on first page, and scrape the lot_size, year, ac, heater, garage, price/sqft info
    home_values = soup.findAll('span', attrs = {'class':'ds-body ds-home-fact-value'})
    hv_ls = []
    for value in home_values:
        hv_ls.append(value.get_text())   
    if len(hv_ls) == 0:
        hv_ls = ['None']*7
    #print(hv_ls)   
    lotsize.append(hv_ls[6])
    hoa.append(hv_ls[5])
    year.append(hv_ls[1])
    ac.append(hv_ls[3])
    heat.append(hv_ls[2])
    park.append(hv_ls[4])
    price_sqft.append(hv_ls[-1])

    #scrape school info    
    schools = soup.findAll('div', attrs = {'class':'ds-nearby-schools-list'})
    if len(schools) == 0:
        scl = ['None', 'None', 'None']
        scl_score = ['None', 'None', 'None']
    elif len(list(schools[0].children)) == 0:
        scl = ['None', 'None', 'None']
        scl_score = ['None', 'None', 'None']
    else:
        scl = ['None', 'None', 'None']
        scl_score = ['None', 'None', 'None']
        for school in schools[0].children:  
            #print(school.span.get_text())
            grade = school.findAll('span')[3].get_text()
            if grade == 'K-6' or grade == 'K-5' or grade == 'K-8':
                scl[0] = school.a.get_text()
                scl_score[0] = school.span.get_text()
            if grade == '7-8' or grade == '6-8':
                scl[1] = school.a.get_text()
                scl_score[1] = school.span.get_text()
            if grade == '9-12':
                scl[2] = school.a.get_text()
                scl_score[2] = school.span.get_text()
    elem.append(scl[0])
    mid.append(scl[1])
    high.append(scl[2])
    elem_s.append(scl_score[0])
    mid_s.append(scl_score[1])
    high_s.append(scl_score[2])
    
    #scrape Neighborhood info
    
    nbs = soup.findAll('li', attrs = {'class':'ds-data-view-item'})
    for nb in soup.findAll(string=re.compile("^Neighborhood:")):
        neighborhood.append(nb.split(':')[1])


IndexError: list index out of range

In [ ]:
len(hv_ls)

In [ ]:
len(neighborhood)

In [ ]:
#add a columns to the empty dataframe
house_df['bds']=bds
house_df['bath']=bath
house_df['areas']=areas
house_df['lot_size']=lotsize
house_df['year_built']=year
house_df['a/c']=ac
house_df['heater']=heat
house_df['garage']=park
house_df['price/sqft']=price_sqft
house_df['elem_score']=elem_s
house_df['middle_score']=mid_s
house_df['high_score']=high_s
house_df['elem_school'] = elem
house_df['middle_school'] = mid
house_df['high_school'] = high
house_df['pricllllle'] = price_ls
house_df['HOA'] = hoa
house_df['address']=adr_ls
house_df['neighborhood']=neighborhood

In [ ]:
house_df.head()

In [9]:
#save data on CSV file
house_df.to_csv('market_houses_Newark.csv',sep=',')

In [10]:
#find all links of 80 sold houses on the first 2 page of the search, sorted by newest to oldest
soldpage_urls = ['https://www.zillow.com/newark-ca/sold/house_type/?searchQueryState={%22pagination%22:{},%22mapBounds%22:{%22west%22:-122.16536934228516,%22east%22:-121.89551765771485,%22south%22:37.436669756614464,%22north%22:37.60533202072703},%22regionSelection%22:[{%22regionId%22:49611,%22regionType%22:6}],%22isMapVisible%22:true,%22mapZoom%22:12,%22filterState%22:{%22sortSelection%22:{%22value%22:%22globalrelevanceex%22},%22isForSaleByAgent%22:{%22value%22:false},%22isForSaleByOwner%22:{%22value%22:false},%22isNewConstruction%22:{%22value%22:false},%22isForSaleForeclosure%22:{%22value%22:false},%22isComingSoon%22:{%22value%22:false},%22isAuction%22:{%22value%22:false},%22isPreMarketForeclosure%22:{%22value%22:false},%22isPreMarketPreForeclosure%22:{%22value%22:false},%22isRecentlySold%22:{%22value%22:true},%22isCondo%22:{%22value%22:false},%22isMultiFamily%22:{%22value%22:false},%22isManufactured%22:{%22value%22:false},%22isLotLand%22:{%22value%22:false},%22isTownhouse%22:{%22value%22:false},%22isApartment%22:{%22value%22:false}},%22isListVisible%22:true}',
                 'https://www.zillow.com/newark-ca/sold/house_type/2_p/?searchQueryState={%22pagination%22:{%22currentPage%22:2},%22usersSearchTerm%22:%22Newark,%20CA%22,%22mapBounds%22:{%22west%22:-122.16536934228516,%22east%22:-121.89551765771485,%22south%22:37.436669756614464,%22north%22:37.60533202072703},%22regionSelection%22:[{%22regionId%22:49611,%22regionType%22:6}],%22mapZoom%22:12,%22isMapVisible%22:true,%22filterState%22:{%22isCondo%22:{%22value%22:false},%22isMultiFamily%22:{%22value%22:false},%22isManufactured%22:{%22value%22:false},%22isLotLand%22:{%22value%22:false},%22isTownhouse%22:{%22value%22:false},%22isApartment%22:{%22value%22:false},%22isRecentlySold%22:{%22value%22:true},%22isForSaleByAgent%22:{%22value%22:false},%22isForSaleByOwner%22:{%22value%22:false},%22isNewConstruction%22:{%22value%22:false},%22isComingSoon%22:{%22value%22:false},%22isAuction%22:{%22value%22:false},%22isForSaleForeclosure%22:{%22value%22:false},%22isPreMarketForeclosure%22:{%22value%22:false},%22isPreMarketPreForeclosure%22:{%22value%22:false}},%22isListVisible%22:true}']
sold_urllist = []
for url in soldpage_urls:
    req = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text = urlopen(req).read().decode(encoding="utf-8", errors='ignore')
    soup = BS(text, 'html.parser')
    for link in soup.findAll('a',attrs = {'class':'list-card-link', 'tabindex':"0"}, href = True):
        
        sold_urllist.append(link['href'])

In [11]:
len(sold_urllist[40:80])

40

In [12]:
soldhouse_df = pd.DataFrame()

In [13]:
#informations on the first listing page of all 40 houses
bds_sold = []
bath_sold = []
areas_sold = []
price_ls_sold = []
adr_ls_sold = []
sold_dates = []
for url in soldpage_urls:
    req_sold = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text_sold = urlopen(req_sold).read().decode(encoding="utf8", errors='ignore')
    soup_sold = BS(text_sold, 'html.parser')
    
    soldhouse_prices = soup_sold.findAll('div', attrs = {'class':'list-card-price'})
    for price in soldhouse_prices:
        price_ls_sold.append(price.get_text())


    
    #extract room info
    lists_sold = soup_sold.findAll('ul', attrs = {'class':'list-card-details'})
    details_sold = soup_sold.findAll('ul', attrs = {'class':'list-card-details'})
    #print(len(details))
    for detail in details_sold:
        #print(detail.li)
        bds_sold.append(detail.li.get_text())
        bath_sold.append(detail.li.next_sibling.get_text())
        areas_sold.append(detail.li.next_sibling.next_sibling.get_text())
    
    #extract address info
    addresses_sold = soup_sold.findAll('address', attrs = {'class':'list-card-addr'})
    for address in addresses_sold:
        adr_ls_sold.append(address.get_text())
        
    #extract sold dates
    dates = soup_sold.findAll('div', attrs={'class':'list-card-variable-text list-card-img-overlay'})
    for date in dates:
        sold_dates.append(date.get_text().split(' ')[1])

In [23]:
len(sold_dates)

80

In [24]:
#details after opening each url of a house
lotsize_sold1 = []
year_sold1 = []
ac_sold1 = []
heat_sold1 = []
park_sold1 = []

hoa_sold1 = []

elem_sold1 = []
mid_sold1 =[]
high_sold1=[]

elem_s_sold1=[]
mid_s_sold1=[]
high_s_sold1=[]

neighborhood_sold1 = []

for url in sold_urllist[0:40]:
    #print(url)
    req_sold1 = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text_sold1 = urlopen(req_sold1).read().decode(encoding="utf-8", errors='ignore')
    soup_sold1 = BS(text_sold1, 'html.parser')
    
    #iterate all house url on first page, and scrape the lot_size, year, ac, heater, garage, price/sqft info
    home_values_sold1 = soup_sold1.findAll('div', attrs = {'class':'fact-value'})
    hv_ls_sold1 = []
    for value in home_values_sold1:
        hv_ls_sold1.append(value.get_text())   
    if len(hv_ls_sold1) == 0:
        hv_ls_sold1 = ['None']*7
    #print(hv_ls_sold1)   
    lotsize_sold1.append(hv_ls_sold1[6])
    hoa_sold1.append(hv_ls_sold1[5])
    year_sold1.append(hv_ls_sold1[1])
    ac_sold1.append(hv_ls_sold1[3])
    heat_sold1.append(hv_ls_sold1[2])
    park_sold1.append(hv_ls_sold1[4])

    

    #scrape school info    
    schools1 = soup_sold1.findAll('li', attrs = {'class':'nearby-school assigned-school clearfix'})
    if len(schools1) == 0:
        scl1 = ['None', 'None', 'None']
        scl_score1 = ['None', 'None', 'None']
    else:
        scl1 = ['None', 'None', 'None']
        scl_score1 = ['None', 'None', 'None']
        for school in schools1:  
            #print(school.span.get_text())
            grade = school.findAll('div')[5].get_text()
            if grade == 'K-6' or grade == 'K-5' or grade == 'K-8':
                scl1[0] = school.a.get_text()
                scl_score1[0] = school.span.get_text()
            if grade == '7-8' or grade == '6-8':
                scl1[1] = school.a.get_text()
                scl_score1[1] = school.span.get_text()
            if grade == '9-12':
                scl1[2] = school.a.get_text()
                scl_score1[2] = school.span.get_text()
    elem_sold1.append(scl1[0])
    mid_sold1.append(scl1[1])
    high_sold1.append(scl1[2])
    elem_s_sold1.append(scl_score1[0])
    mid_s_sold1.append(scl_score1[1])
    high_s_sold1.append(scl_score1[2])
    
    #scrape Neiborhood info
    for nb in soup_sold1.findAll(string=re.compile("^Neighborhood:")):
        neighborhood_sold1.append(nb.split(':')[1])


In [25]:
len(neighborhood_sold1)

40

In [ ]:
#details after opening each url of a house
lotsize_sold2 = []
year_sold2 = []
ac_sold2 = []
heat_sold2 = []
park_sold2 = []

hoa_sold2 = []
bath_sold2 = []

elem_sold2 = []
mid_sold2 =[]
high_sold2=[]

elem_s_sold2=[]
mid_s_sold2=[]
high_s_sold2=[]

neighborhood_sold2 = []

for url in sold_urllist[40:80]:
    #print(url)
    req_sold2 = Request(url, headers = {'user-Agent':'Mozilla/5.0'})
    text_sold2 = urlopen(req_sold2).read().decode(encoding="utf-8", errors='ignore')
    soup_sold2 = BS(text_sold2, 'html.parser')
    
    #iterate all house url on first page, and scrape the lot_size, year, ac, heater, garage, price/sqft info
    home_values_sold2 = soup_sold2.findAll('div', attrs = {'class':'fact-value'})
    hv_ls_sold2 = []
    for value in home_values_sold2:
        hv_ls_sold2.append(value.get_text())   
    if len(hv_ls_sold2) == 0:
        hv_ls_sold2 = ['None']*7
    #print(hv_ls_sold2)   
    lotsize_sold2.append(hv_ls_sold2[6])
    hoa_sold2.append(hv_ls_sold2[5])
    year_sold2.append(hv_ls_sold2[1])
    ac_sold2.append(hv_ls_sold2[3])
    heat_sold2.append(hv_ls_sold2[2])
    park_sold2.append(hv_ls_sold2[4])

    

    #scrape school info    
    schools2 = soup_sold2.findAll('li', attrs = {'class':'nearby-school assigned-school clearfix'})
    if len(schools2) == 0:
        scl2 = ['None', 'None', 'None']
        scl_score2 = ['None', 'None', 'None']
    else:
        scl2 = ['None', 'None', 'None']
        scl_score2 = ['None', 'None', 'None']
        for school in schools2:  
            #print(school.span.get_text())
            grade = school.findAll('div')[5].get_text()
            if grade == 'K-6' or grade == 'K-5' or grade == 'K-8':
                scl2[0] = school.a.get_text()
                scl_score2[0] = school.span.get_text()
            if grade == '7-8' or grade == '6-8':
                scl2[1] = school.a.get_text()
                scl_score2[1] = school.span.get_text()
            if grade == '9-12':
                scl2[2] = school.a.get_text()
                scl_score2[2] = school.span.get_text()
    elem_sold2.append(scl2[0])
    mid_sold2.append(scl2[1])
    high_sold2.append(scl2[2])
    elem_s_sold2.append(scl_score2[0])
    mid_s_sold2.append(scl_score2[1])
    high_s_sold2.append(scl_score2[2])
    
    #scrape Neiborhood info
    for nb in soup_sold2.findAll(string=re.compile("^Neighborhood:")):
        neighborhood_sold2.append(nb.split(':')[1])

In [ ]:
len(neighborhood_sold2)

In [ ]:
lotsize_sold=lotsize_sold1+lotsize_sold2
year_sold=year_sold1+year_sold2
ac_sold=ac_sold1+ac_sold2
heat_sold=heat_sold1+heat_sold2
park_sold=park_sold1+park_sold2
hoa_sold=hoa_sold1+hoa_sold2
elem_sold=elem_sold1+elem_sold2
mid_sold=mid_sold1+mid_sold2
high_sold=high_sold1+high_sold2
elem_s_sold=elem_s_sold1+elem_s_sold2
mid_s_sold=mid_s_sold1+mid_s_sold2
high_s_sold=high_s_sold1+high_s_sold2
neighborhood_sold=neighborhood_sold1+neighborhood_sold2

In [ ]:
#add a columns to the empty dataframe
soldhouse_df['bds']=bds_sold
soldhouse_df['bath']=bath_sold
soldhouse_df['areas']=areas_sold
soldhouse_df['lot_size']=lotsize_sold
soldhouse_df['year_built']=year_sold
soldhouse_df['a/c']=ac_sold
soldhouse_df['heater']=heat_sold
soldhouse_df['garage']=park_sold
soldhouse_df['elem_score']=elem_s_sold
soldhouse_df['middle_score']=mid_s_sold
soldhouse_df['high_score']=high_s_sold
soldhouse_df['elem_school'] = elem_sold
soldhouse_df['middle_school'] = mid_sold
soldhouse_df['high_school'] = high_sold
soldhouse_df['sold_price'] = price_ls_sold
soldhouse_df['HOA'] = hoa_sold
soldhouse_df['address']=adr_ls_sold
soldhouse_df['date_sold']=sold_dates
soldhouse_df['neighborhood']=neighborhood_sold

In [ ]:
len(bath_sold)

In [ ]:
soldhouse_df.head()

In [ ]:
len(bath_sold)

In [18]:
soldhouse_df.to_csv('sold_houses_Newark.csv',sep=',')